In [2]:
import spacy
from spacy.training import Example
from spacy.util import minibatch, compounding
import random
import pandas as pd

In [3]:
# Load dataset
def process_csv_for_spacy(file_path):
    df = pd.read_csv(file_path)

    train_data = []

    for _, row in df.iterrows():
        sentence = row['Sentence']
        label = row['Label']
        start = row['Start']
        end = row['End']

        # Nếu start và end đều bằng 0, xử lý cho textcat
        if start == 0 and end == 0:
            if 'cats' not in locals():
                cats = {}
            cats[label] = 1
        
        # Ngược lại, xử lý cho NER
        else:
            if 'entities' not in locals():
                entities = []
            entities.append((start, end, label))
        
        # Sau khi đi qua tất cả các cột, thêm vào train_data
        if 'entities' in locals() or 'cats' in locals():
            annotations = {}
            if 'entities' in locals():
                annotations['entities'] = entities
            if 'cats' in locals():
                annotations['cats'] = cats
            train_data.append((sentence, annotations))
            # Xóa các biến tạm thời để chuẩn bị cho dòng dữ liệu mới
            if 'entities' in locals():
                del entities
            if 'cats' in locals():
                del cats

    return train_data

train_data = process_csv_for_spacy('spacy_dataset.csv')

for data in train_data:
    print(data)

('Please set a task in the Artemis project, about creating a user feedback system. This is an important task but not urgent.', {'entities': [(25, 32, 'PROJECT')]})
('Please set a task in the Artemis project, about creating a user feedback system. This is an important task but not urgent.', {'cats': {'GROUPTASK': 1}})
('Please set a task in the Artemis project, about creating a user feedback system. This is an important task but not urgent.', {'entities': [(48, 79, 'TASK')]})
('Please set a task in the Artemis project, about creating a user feedback system. This is an important task but not urgent.', {'cats': {'MEDIUM': 1}})
('Please set a task in the Artemis project, about creating a user feedback system. This is an important task but not urgent.', {'cats': {'PENDING': 1}})
('Create task to verify database integrity after recent updates. This is a star priority.', {'entities': [(15, 40, 'TASK')]})
('Create task to verify database integrity after recent updates. This is a star priority.